In [2]:
# Imports + Dataset Preparation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score

# Load dataset
data = load_diabetes()
X, y = data.data, data.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data prepared: Train shape =", X_train_scaled.shape, ", Test shape =", X_test_scaled.shape)


Data prepared: Train shape = (353, 10) , Test shape = (89, 10)


In [4]:
# Ridge Hyperparameter Tuning
ridge = Ridge()

# Define parameter grid
ridge_params = {"alpha": np.logspace(-3, 3, 50)}  # from 0.001 to 1000

# GridSearch with 5-fold CV
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5, scoring="r2")
ridge_grid.fit(X_train_scaled, y_train)

print("Best Ridge alpha:", ridge_grid.best_params_["alpha"])
print("Best Ridge CV R²:", ridge_grid.best_score_)


Best Ridge alpha: 44.98432668969444
Best Ridge CV R²: 0.4564057908977427


In [6]:
# Lasso Hyperparameter Tuning
lasso = Lasso(max_iter=10000)

# Define parameter grid
lasso_params = {"alpha": np.logspace(-3, 1, 50)}  # from 0.001 to 10

# GridSearch with 5-fold CV
lasso_grid = GridSearchCV(lasso, lasso_params, cv=5, scoring="r2")
lasso_grid.fit(X_train_scaled, y_train)

print("Best Lasso alpha:", lasso_grid.best_params_["alpha"])
print("Best Lasso CV R²:", lasso_grid.best_score_)


Best Lasso alpha: 1.8420699693267144
Best Lasso CV R²: 0.4556950145191811


In [8]:
# Elastic Net Hyperparameter Tuning
elastic_net = ElasticNet(max_iter=10000)

# Define parameter grid (alpha + l1_ratio both tuned)
elastic_params = {
    "alpha": np.logspace(-3, 1, 20),   # 0.001 → 10
    "l1_ratio": np.linspace(0.1, 0.9, 9)  # mix between L1 & L2
}

# GridSearchCV with 5-fold cross validation
elastic_grid = GridSearchCV(elastic_net, elastic_params, cv=5, scoring="r2")
elastic_grid.fit(X_train_scaled, y_train)

print("Best ElasticNet params:", elastic_grid.best_params_)
print("Best ElasticNet CV R²:", elastic_grid.best_score_)


Best ElasticNet params: {'alpha': 0.20691380811147903, 'l1_ratio': 0.30000000000000004}
Best ElasticNet CV R²: 0.4564109445802577


In [12]:
# Final Comparison on Test Set
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)


# Best tuned models
ridge_best = ridge_grid.best_estimator_
lasso_best = lasso_grid.best_estimator_
elastic_best = elastic_grid.best_estimator_

# Evaluate all
models_final = {
    "Linear Regression": lin_reg,
    "Ridge": ridge_best,
    "Lasso": lasso_best,
    "Elastic Net": elastic_best
}

print("\n🧾 Final Test R² Scores:")
for name, model in models_final.items():
    y_pred = model.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    print(f"{name}: {r2:.4f}")



🧾 Final Test R² Scores:
Linear Regression: 0.4526
Ridge: 0.4608
Lasso: 0.4716
Elastic Net: 0.4615
